![](pictures/openvino_start.png)

# Section 0: Register and Set Up Environment

The first step of the workshop is initializing the OpenVINO™ environment in this Jupyter notebook. 
The OpenVINO™ 2020.1 package have been installed to `intel/openvino/` already.

To initialize the OpenVINO™ environment, run the `intel/openvino/bin/setupvars.sh` script.
If the prerequisite steps have been done right, you will see the output: 

```
[setupvars.sh] OpenVINO environment initialized
OpenVINO Inference Engine version is: ....
```

In [ ]:
!bash ~/intel/openvino_2021/bin/setupvars.sh

from openvino import inference_engine as ie
print('OpenVINO Inference Engine version: {}'.format(ie.__version__))

# Agenda

##  1. [Introduction](#s1)

##  2. [What is SSD MobileNet V2?](#s2)

##  3. [Where Can I Find the Model?](#s3)

##  4. [Infer SSD MobileNet V2 on TensorFlow](#s4)

##  5. [Infer on Real Data on TensorFlow](#s5)

##  6. [OpenVINO™ Overview](#s6)

##  7. [Model Optimizer - Entry to OpenVINO™](#s7)

##  8. [Inference of SSD MobileNet V2 on OpenVINO™ Inference Engine](#s8)

##  9. [Accuracy Checker - OpenVINO&trade; Accuracy Validation Framework](#s9)

## 10. [Quantize the Model to Low Precision](#s10)

## 11. [Post-Trainig Optimization Toolkit](#s11)

## 12. [AccuracyAware Algorithm](#s12)

## 13. [VNNI - Deep Learning Boost](#s13)

## 14. [Get Even Better Performance](#s14)

## 15. [Practice](#s15)

# Section 1: Introduction<a id='s1'></a>

![](pictures/training_vs_inference.png)

![](pictures/about_vino.png)

## Download COCO 2017 calidation dataset 

In [ ]:
! mkdir -p data/datasets/
! curl http://images.cocodataset.org/zips/val2017.zip --output data/datasets/coco.zip
! curl http://images.cocodataset.org/annotations/annotations_trainval2017.zip --output data/datasets/coco_annotations.zip
! unzip data/datasets/coco.zip -d data/datasets/COCO2017
! unzip data/datasets/coco_annotations.zip -d data/datasets/COCO2017

In [ ]:
# mostly for working with paths: os.path
import os

# working with arrays
import numpy as np 

# path to data for the workshop
WORKSHOP_DATA_PATH = os.path.join('.', 'data')

## Inference in 6 Lines

Once [OpenVINO™](https://docs.openvinotoolkit.org/) is installed, you can run an inference:

In [ ]:
from openvino import inference_engine as ie

# Create an instance of the OpenVINO Inference Engine Core 
# This is the key module of the OpenVINO Inference Engine
ie_core = ie.IECore()

# Read a network from the Intermediate Representation (IR)
network = ie_core.read_network(os.path.join(WORKSHOP_DATA_PATH, 'model.xml'), 
                               os.path.join(WORKSHOP_DATA_PATH, 'model.bin'))

# Find inputs of the model
input_layer = next(iter(network.input_info))

# Get input shape of the network
input_shape = network.input_info[input_layer].input_data.shape

# Load the network that was read from the Intermediate Representation (IR) 
# to the CPU device 
network_loaded_on_device = ie_core.load_network(network=network, device_name='CPU')

# Start an inference of the loaded network and return output data
network_loaded_on_device.infer(inputs={input_layer: np.random.rand(*input_shape)})

For more information, go to references of [OpenVINO Inference Engine Python API](https://docs.openvinotoolkit.org/latest/ie_python_api/annotated.html).

![](./pictures/infer.PNG)

# Section 2: What is SSD MobileNet V2?<a id='s2'></a>

![](pictures/mobileNet-SSD-network-architecture.png)

The `ssd_mobilenet_v2_coco` model is a [Single-Shot multibox Detection (SSD)](https://arxiv.org/pdf/1801.04381.pdf) network for object detection. The model has been trained from the Common Objects in Context (COCO) image dataset.

The model input is a blob that consists of a single image of `1x3x300x300` in the `RGB` order.

The model output is a typical vector containing the tracked object data. Note that the `class_id` data is now significant and should be used to determine the classification for any detected object.

Model outputs:

1. Classifier, name - `detection_classes`, contains predicted bounding boxes classes in range `[1, 91]`. The model was trained on Microsoft\* COCO dataset version with 90 categories of objects.
2. Probability, name - `detection_scores`, contains probability of detected bounding boxes.
3. Detection box, name - `detection_boxes`, contains detection boxes coordinates in format `[y_min, x_min, y_max, x_max]`, where (`x_min`, `y_min`)  are coordinates top left corner, (`x_max`, `y_max`) are coordinates of the right bottom corner. Coordinates are rescaled to the input image size.
4. Detections number, name - `num_detections`, contains the number of predicted detection boxes.

# Section 3: Where Can I Find the Model?<a id='s3'></a>

With the OpenVINO™ toolkit, you can easily download models from the [Intel&reg; Open Model Zoo](https://github.com/opencv/open_model_zoo), which contains both public open-sourse models from different frameworks (TensorFlow\*, Caffe\*, MxNet\*, PyTorch\* and others) and models created at Intel&reg;.

To see all available models, run the `downloader.py` script with the `--print_all` parameter. 

In [ ]:
!python3 ${INTEL_OPENVINO_DIR}/deployment_tools/open_model_zoo/tools/downloader/downloader.py --print_all

Let's try to download an object-detection model called `ssd_mobilenet_v2_coco` using the [Model Downloader](https://github.com/opencv/open_model_zoo/tree/master/tools/downloader).

In [ ]:
!python3 ${INTEL_OPENVINO_DIR}/deployment_tools/open_model_zoo/tools/downloader/downloader.py \
--name ssd_mobilenet_v2_coco \
--output_dir ./data

As we can see, the Model Downloader can load not only publicly famous model, but also various models created at Intel for a range of tasks.

![](pictures/models.png)

Model Downloader downloaded the model to the following directory: `data/public/ssd_mobilenet_v2_coco/ssd_mobilenet_v2_coco_2018_03_29`.

In [ ]:
!ls data/public/ssd_mobilenet_v2_coco/ssd_mobilenet_v2_coco_2018_03_29

# Section 4: Infer SSD MobileNet V2 in TensorFlow<a id='s4'></a> 

In [ ]:
import os
import tensorflow as tf
from tensorflow.gfile import GFile

# Path to the TensorFlow model
model = os.path.join('data', 'public', 
                     'ssd_mobilenet_v2_coco', 
                     'ssd_mobilenet_v2_coco_2018_03_29', 
                     'frozen_inference_graph.pb')

# SSD MobileNet v2 contains following output nodes
output_names = ['detection_classes:0','detection_scores:0', 'detection_boxes:0', 'num_detections:0']

# Create a graph
graph = tf.Graph()

# Create graph definitions
graph_def = tf.GraphDef()

# Read model to the graph definitions
with open(model, "rb") as model_file:
    graph_def.ParseFromString(model_file.read())


with graph.as_default():
    # Import the graph definitions to TensorFlow
    tf.import_graph_def(graph_def, name='')
    # Get tensors for output nodes
    output_tensors = [graph.get_tensor_by_name(layer_name) for layer_name in output_names] 

    with tf.Session(graph=graph) as session:
        # Infer the model for random datates
        print(session.run(output_tensors, feed_dict = {'image_tensor:0' : np.random.rand(1, 300, 300, 3)}))

![](pictures/inference_tf.PNG)

# Section 5: Infer on Real Data in TensorFlow<a id='s5'></a>

To run the TensorFlow `ssd_mobilenet_v2_coco` model, we need some utility functions and constant values:

In [ ]:
import logging as log
import os
import sys

# Initialize logging
log.basicConfig(format="[ %(levelname)s ] %(message)s", level=log.INFO, stream=sys.stdout)

# Define how many times we run inference to get better performance
NUM_RUNS = 1 
# Number of images for one inference
BATCH = 1

# Contains all data for the workshop
WORKSHOP_DATA_PATH = os.path.join(os.getcwd(), 'data')

# Path to a test image
IMAGE = os.path.join(WORKSHOP_DATA_PATH, 'images', 'input', 'cats.jpg')

# Path to the downloaded TensorFlow image
SSD_ASSETS = os.path.join(WORKSHOP_DATA_PATH, 'public', 'ssd_mobilenet_v2_coco')

# Path to the downloaded frozen TensorFlow image
TF_MODEL = os.path.join(SSD_ASSETS,
                        'ssd_mobilenet_v2_coco_2018_03_29', 
                        'frozen_inference_graph.pb')

# Create output directory:
OUTPUT_PATH = os.path.join(WORKSHOP_DATA_PATH, 'images', 'output')
os.makedirs(OUTPUT_PATH, exist_ok=True)

# Path to the resulting TensorFlow image
TF_RESULT_IMAGE = os.path.join(OUTPUT_PATH, 'tensorflow_output.png')

# Path to the Inference Engine FP32 model
IE_MODEL_FP32_XML = os.path.join(SSD_ASSETS, 'FP32', 'ssd_mobilenet_v2_coco.xml')
IE_MODEL_FP32_BIN = os.path.join(SSD_ASSETS, 'FP32', 'ssd_mobilenet_v2_coco.bin')

# Path to the Inference Engine INT8 model optimized with the Default algorithm
IE_MODEL_DEFAULT_INT8_XML = os.path.join(SSD_ASSETS, 'INT8', 'default', 'optimized', 'ssd_mobilenet_v2_coco.xml')
IE_MODEL_DEFAULT_INT8_BIN = os.path.join(SSD_ASSETS, 'INT8', 'default', 'optimized', 'ssd_mobilenet_v2_coco.bin')

# Path to the Inference Engine INT8 model optimized  with the AccuracyAware algorithm
IE_MODEL_AA_INT8_XML = os.path.join(SSD_ASSETS, 'INT8', 'acuracy_aware', 'optimized', 'ssd_mobilenet_v2_coco.xml')
IE_MODEL_AA_INT8_BIN = os.path.join(SSD_ASSETS, 'INT8', 'acuracy_aware', 'optimized', 'ssd_mobilenet_v2_coco.bin')

# Path to the resulting InferenceEngine image
IE_RESULT_IMAGE = os.path.join(OUTPUT_PATH, 'inference_engine_output.png')

# Path to the combination of the resulting TensorFlow and Inference Engine images
COMBO_RESULT_IMAGE = os.path.join(OUTPUT_PATH, 'combo_output.png')

PERFORMANCE = {}

In [ ]:
# Import OpenCV for image processing
import cv2

def read_resize_image(path_to_image: str, width: int, height: int):
    """
    Takes an image and resizes it to the given dimensions.
    """
    # Load the image 
    raw_image = cv2.imread(path_to_image)
    # Return the image resized to the (width, height) format
    return cv2.resize(raw_image, (width, height), interpolation=cv2.INTER_NEAREST)

In [ ]:
# Import required functions from TensorFlow
import tensorflow as tf

import time

def tf_inference(graph: tf.Graph, input_data, input_name: str, outputs_names: list) -> tuple:
    """
    Returns TensorFlow model inference results.
    """
    
    log.info("Running inference with TensorFlow ...")
  
    # Get the input tensor by name
    input_tensor =  graph.get_tensor_by_name('{}:0'.format(input_name))
    
    # Fill input data
    feed_dict = {
        input_tensor: [input_data, ] 
    }

    # Collect output tensors
    output_tensors = []
    
    for output_name in outputs_names:
        tensor = graph.get_tensor_by_name('{}:0'.format(output_name))
        output_tensors.append(tensor)
    
    # Run inference and get performance
    log.info("Running tf.Session")
    with graph.as_default():
        with tf.Session(graph=graph) as session:
            inference_start = time.time()
            outputs = session.run(output_tensors, feed_dict=feed_dict)
            inference_end = time.time()
    
    # Collect inference results
    res = dict(zip(outputs_names, outputs))
    
    log.info("TensorFlow reference collected successfully")
    
    return res, inference_end - inference_start

In [ ]:
import tensorflow as tf
import numpy as np

def tf_main(path_to_pb_model: str, 
            path_to_original_image: str, 
            number_inference: int = 1):
    """
    Entrypoint to infer with TensorFlow.
    """
    log.info('COMMON: image preprocessing')
    
    # Size of the image is 300x300 pixels, 3 channels in the RGB format
    width = 300
    
    image_shape = (300, 300, 3)
    
    resized_image = read_resize_image(path_to_original_image, width, width)
    
    reshaped_image = np.reshape(resized_image, image_shape)
    
    log.info('Current shape: {}'.format(reshaped_image.shape))

    log.info('TENSORFLOW SPECIFIC: Loading a model with TensorFlow')
    
    graph = tf.Graph()
    
    graph_def = tf.GraphDef()

    with open(path_to_pb_model, "rb") as model_file:
        graph_def.ParseFromString(model_file.read())

    with graph.as_default():
        tf.import_graph_def(graph_def, name='')

    log.info("TensorFlow graph was created")
    
    # We use SSD MobileNet V2 and we know the name of the input 
    input_layer = 'image_tensor'
    
    # And we know names of outputs
    output_layers = ['num_detections', 'detection_classes', 'detection_scores', 'detection_boxes']
    
    collected_inference_time = []
    
    for run in range(number_inference):
        raw_results, inference_time = tf_inference(graph, reshaped_image, input_layer, output_layers)
        collected_inference_time.append(inference_time)
    
    tensorflow_average_inference_time = sum(collected_inference_time) / number_inference
    
    log.info('TENSORFLOW SPECIFIC: Plain inference finished')

    return raw_results, tensorflow_average_inference_time

In [ ]:
# Import the Image class from display to show an image
from IPython.display import Image
# Show the image in the notebok
Image(filename=IMAGE)

## Infer the Model on a Real Image

In [ ]:
framework = 'TF'
device = 'CPU'
name = '{f} on {d}'.format(f=framework, d=device)

tensorflow_fps_collected = []

# Run inference in TensorFlow
tensorflow_predictions, tensorflow_average_inference_time = tf_main(TF_MODEL, IMAGE, number_inference=NUM_RUNS)
    
log.info(f'Inference Time of SSD MobileNet V2 {name} is {tensorflow_average_inference_time} seconds')

# Calculate FPS from inference time
tensorflow_average_fps = 1 / tensorflow_average_inference_time

log.info(f'{name} frames per second (FPS): {tensorflow_average_fps}')

In [ ]:
print(tensorflow_predictions['num_detections']) # get number of detected objects
print(tensorflow_predictions['detection_classes'][0])# get predicted classes IDs
print(tensorflow_predictions['detection_scores'][0]) # get probabilities for predicted classes
print(tensorflow_predictions['detection_boxes'][0]) # get boxes for predicted objects

In [ ]:
# Import utility functions to process images from TensorFlow and draw images
from utils import parse_od_output, draw_image

# Import the Image class from display to show an image
from IPython.display import Image

processd_tensorflow_predictions = parse_od_output(tensorflow_predictions)
draw_image(IMAGE, processd_tensorflow_predictions, TF_RESULT_IMAGE)

# Show the image in the notebok
Image(filename=TF_RESULT_IMAGE)

# Section 6: [OpenVINO&trade;](https://docs.openvinotoolkit.org/) Overview<a id='s6'></a>

![](./pictures/infer.PNG)

![](pictures/openvino_toolkit.png)

![](pictures/additional_tools.png)

# Section 7: [Model Optimizer](https://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) - Entry to OpenVINO&trade;<a id='s7'></a>

![](pictures/model_optimizer.png)

 Let's convert the TensorFlow model to the IR format:

In [ ]:
!python3 ${INTEL_OPENVINO_DIR}/deployment_tools/model_optimizer/mo.py \
--output_dir=data/public/ssd_mobilenet_v2_coco/FP32 \
--reverse_input_channels \
--model_name=ssd_mobilenet_v2_coco \
--transformations_config=${INTEL_OPENVINO_DIR}/deployment_tools/model_optimizer/extensions/front/tf/ssd_v2_support.json \
--tensorflow_object_detection_api_pipeline_config=data/public/ssd_mobilenet_v2_coco/ssd_mobilenet_v2_coco_2018_03_29/pipeline.config \
--output=detection_classes,detection_scores,detection_boxes,num_detections \
--input_model=data/public/ssd_mobilenet_v2_coco/ssd_mobilenet_v2_coco_2018_03_29/frozen_inference_graph.pb

![](./pictures/openvino_support.png)

We can view the Intermediate Representation of the SSD MobileNet V2:

In [ ]:
!cat data/public/ssd_mobilenet_v2_coco/FP32/ssd_mobilenet_v2_coco.xml

# Section 8: Inference of SSD MobileNet V2 on [OpenVINO™ Inference Engine](https://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html)<a id='s8'></a> 

In [ ]:
from openvino.inference_engine import IECore, IENetwork
import numpy as np
import time

def ie_inference(path_to_model_xml: str, path_to_model_bin: str, path_to_original_image: str, device='CPU', batch=1):
    """
    Entrypoint to infer with the OpenVINO Inference Engine
    """

    # Now let's create the IECore() entity 
    log.info("Creating Inference Engine Core")   
    ie = IECore()

    # First, create a network (Note: you need to provide model in the IR previously converted with Model Optimizer)
    log.info("Reading IR...")
    net = ie.read_network(model=path_to_model_xml, weights=path_to_model_bin)

    # Get input and output blob of the network
    input_blob = next(iter(net.input_info))
    out_blob = next(iter(net.outputs))

    # Reshape the network to the needed batch
    _, c, h, w = net.input_info[input_blob].input_data.shape
    
    net.reshape({input_blob: (batch, c, h, w)})
    
    # Resize the image 
    log.info('COMMON: image preprocessing')
    image = read_resize_image(path_to_original_image, h, w)
    
    # Now we load the network to the plugin
    log.info("Loading IR to the plugin...")
    exec_net = ie.load_network(network=net, device_name=device)

    del net

    labels_map = None
    
    # Read and preprocess the input image
    image = image[..., ::-1]
    in_frame = image.transpose((2, 0, 1))  # Change data layout from HWC to CHW
    batched_frame = np.array([in_frame for _ in range(batch)])
    log.info('Current shape: {}'.format(batched_frame.shape))

    # Now we run an inference on the target device
    inference_start = time.time()
    res = exec_net.infer(inputs={input_blob: batched_frame})
    inference_end = time.time()

    log.info('INFERENCE ENGINE SPECIFIC: no post-processing')

    return res[out_blob], inference_end - inference_start

In [ ]:
def ie_main(xml:str, bin:str, device:str, postfix: str = ''):
    name = '{f} {p} on {d}'.format(f='IE', p=postfix, d=device)

    inference_engine_fps_collected = []

    for i in range(NUM_RUNS):
        # Run an inference on OpenVINO Inference Engine
        predictions, inference_time = ie_inference(xml, bin,
                                                   IMAGE,
                                                   device,
                                                   batch=BATCH)
        
        log.info('Inference Time of SSD MobileNet V2 {}: {} seconds'.format(name, inference_time))
        # Calculate FPS from inference time
        inference_engine_fps = 1 / inference_time
        
        inference_engine_fps_collected.append(inference_engine_fps)

    # Calculate the average FPS for all inferences
    inference_engine_avg_fps = (sum(inference_engine_fps_collected) * BATCH) / (NUM_RUNS)
    
    PERFORMANCE[name] = inference_engine_avg_fps

    log.info('{} frames per second (FPS): {}'.format(name, inference_engine_avg_fps))
    
    return inference_engine_avg_fps, predictions

In [ ]:
device = 'CPU'

# Run the inference 
inference_engine_average_fps, inference_engine_predictions = ie_main(IE_MODEL_FP32_XML, 
                                                                     IE_MODEL_FP32_BIN, 
                                                                     device)

In [ ]:
inference_engine_predictions[0] # get data for the image

In [ ]:
from utils import parse_od_output, draw_image

draw_image(IMAGE, inference_engine_predictions, IE_RESULT_IMAGE, color=(255, 0, 0))

In [ ]:
# Import the Image class from display to show an image
from IPython.display import Image

# Show the image in the notebok
Image(filename=IE_RESULT_IMAGE)

In [ ]:
# Import functions from Matplotlib to show barcharts
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec


def show_results_interactively(tf_image: str, ie_image: str, combination_image: str, ie_fps:float, tf_fps:float):
    """
    Takes paths to three images and shows them with Matplotlib on one screen.
    """
    _ = plt.figure(figsize=(30, 10))
    gs1 = gridspec.GridSpec(1, 3)
    gs1.update(wspace=0.25, hspace=0.05)

    titles = [
        '(a) TensorFlow',
        '(b) Inference Engine',
        '(c) TensorFlow and Inference Engine\n predictions are identical'
    ]

    for i, path in enumerate([tf_image, ie_image, combination_image]):
        img_resized = cv2.imread(path)
        ax_plot = plt.subplot(gs1[i])
        ax_plot.axis("off")
        addon = ' '
        if i == 1:
            addon += '{:4.3f}'.format(ie_fps) + '(FPS)'
        elif i == 0:
            addon += '{:4.3f}'.format(tf_fps) + '(FPS)'

        ax_plot.text(0.5, -0.5, titles[i] + addon,
                     size=28, ha="center",
                     transform=ax_plot.transAxes)
        ax_plot.imshow(cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB))

    plt.show()

In [ ]:
from utils import draw_image

# Draw inference results from the Inference Engine in the image with TensorFlow inference results
draw_image(TF_RESULT_IMAGE, inference_engine_predictions, COMBO_RESULT_IMAGE, color=(255, 0, 0))

show_results_interactively(tf_image=TF_RESULT_IMAGE,
                           ie_image=IE_RESULT_IMAGE,
                           combination_image=COMBO_RESULT_IMAGE,
                           ie_fps=inference_engine_average_fps,
                           tf_fps=tensorflow_average_fps)

In [ ]:
from utils import show_performance

performance_data = {
    'TF on CPU': tensorflow_average_fps,
    'IE on CPU': inference_engine_average_fps
}

show_performance(performance_data)

![](pictures/ie_fusing.PNG)

**Oh, this is good - we got the same results in one image. But it is only ONE image. We need to check accuracy on the whole dataset. So how can we do this?**

# Section 9: [Accuracy Checker](https://github.com/opencv/open_model_zoo/tree/master/tools/accuracy_checker) - OpenVINO&trade; Accuracy Validation Framework<a id='s9'></a>

![](pictures/accuracy_check.png)

In [ ]:
# Replace variables to the real path in the Accuracy Checker config:
!WORKSHOP_PATH=$(pwd) envsubst '\${WORKSHOP_PATH}' <data/configs/accuracy_checker_config_tf_template.yml >data/configs/accuracy_checker_config_tf.yml

# Run the Accuracy Checker:
!accuracy_check -c data/configs/accuracy_checker_config_tf.yml

In [ ]:
# Replace variables to the real path in the Accuracy Checker config:
!WORKSHOP_PATH=$(pwd) envsubst '\${WORKSHOP_PATH}' <data/configs/accuracy_checker_config_template.yml >data/configs/accuracy_checker_config.yml

# Run the Accuracy Checker:
!accuracy_check -c data/configs/accuracy_checker_config.yml

In [ ]:
!cat data/configs/accuracy_checker_config.yml

# Section 10: [Quantize the Model to Low Precision](https://docs.openvinotoolkit.org/latest/_compression_algorithms_quantization_README.html)<a id='s10'></a>

![](pictures/quantization.PNG)

![](pictures/quantize.PNG)

# Section 11: [Post-Trainig Optimization Toolkit](https://docs.openvinotoolkit.org/latest/_README.html)<a id='s11'></a>



Post-Training Optimization Toolkit includes standalone command-line tool and Python* API that provide the following key features:

* Two supported post-training quantization algorithms: fast [DefaultQuantization](https://docs.openvinotoolkit.org/latest/_compression_algorithms_quantization_default_README.html) and precise [AccuracyAwareQuantization](https://docs.openvinotoolkit.org/latest/_compression_algorithms_quantization_accuracy_aware_README.html).
as well as multiple experimental methods including global optimization.
* Symmetric and asymmetric quantization schemes. For more details, see the [Quantization](https://docs.openvinotoolkit.org/latest/_compression_algorithms_quantization_README.html) section.
* Compression for different hardware targets such as CPU, GPU.
* Per-channel quantization for Convolutional and Fully-Connected layers.
* Multiple domains: Computer Vision, Recommendation Systems.
* Ability to implement custom calibration pipeline via supported [API](https://docs.openvinotoolkit.org/latest/_sample_README.html).

<br>   

![](pictures/pot.png)

<br>   

JSON files with all info required for calibration:
1. Model parameters (name, path to full precision IR)   
2. Engine Parameters (e.g. preprocessing parameters, dataset path, etc.)    
    2.1 Use accuracy checker .yml config   
    2.2 Define all the required AccuracyChecker parameters directly in the JSON file    
3. Compression parameters (optimization algorithm and its parameters)
  

DefaultQuantization algorithm performs a fast but at the same time accurate INT8 calibration of NNs. It consists of three algorithms that are sequentially applied to a model:
*  ActivationChannelAlignment - Used as a preliminary step before quantization and allows you to align ranges of output activations of Convolutional layers in order to reduce the quantization error.
*  MinMaxQuantization - This is a vanilla quantization method that automatically inserts `FakeQuantize` operations into the model graph based on the specified  target hardware and initializes them
using statistics collected on the calibration dataset.
*  BiasCorrection - Adjusts biases of Convolutional and Fully-Connected layers based on the quantization error of the layer in order to make the overall error unbiased.

In [ ]:
!python3 ${INTEL_OPENVINO_DIR}/deployment_tools/tools/post_training_optimization_toolkit/main.py \
-c data/configs/default/quantization_config.json \
--output-dir data/public/ssd_mobilenet_v2_coco/INT8/default \
--direct-dump

In [ ]:
!cat data/configs/default/quantization_config.json

![](pictures/quantized_ir.PNG)

## [FakeQuantize Layer](https://docs.openvinotoolkit.org/latest/_docs_MO_DG_prepare_model_convert_model_IR_V10_opset1.html#FakeQuantize) <a name="FakeQuantize"></a>

**Name**: *FakeQuantize*

**Category**: *Layer*

**Short description**: *FakeQuantize* layer is element-wise linear quantization of floating-point input values into a discrete set of floating-point values.

**Detailed description**: Input and output ranges as well as the number of levels of quantization are specified by dedicated inputs and attributes. There can be different limits for each element or groups of elements (channels) of the input blobs. Otherwise, one limit applies to all elements. It depends on shape of inputs that specify limits and regular broadcasting rules applied for input blobs. The output of the operator is a floating-point number of the same type as the input blob. In general, there are four values that specify quantization for each element: *input_low*, *input_high*, *output_low*, *output_high*. *input_low* and *input_high* parameters specify the input range of quantization. All input values that are outside this range are clipped to the range before actual quantization. *output_low* and *output_high* specify minimum and maximum quantized values at the output.

**Parameters**: *Quantize* layer parameters are specified in the `data` node, which is a child of the `layer` node.

* **Parameter name**: *levels*

  * **Description**: *levels* is the number of quantization levels.
  * **Range of values**: an integer greater than or equal to 2
  * **Type**: `int`
  * **Default value**: None
  * **Required**: *yes*

**Inputs**:

*   **1**: `X` - multidimensional input blob to quantize. Required.

*   **2**: `input_low` - minimum limit for input value. The shape must be broadcastable to the shape of `X`. Required.

*   **3**: `input_high` - maximum limit for input value. Can be the same as `input_low` for binarization. The shape must be broadcastable to the shape of `X`. Required.

*   **4**: `output_low` - minimum quantized value. The shape must be broadcastable to the shape of `X`. Required.

*   **5**: `output_high` - maximum quantized value. The shape must be broadcastable to the of `X`. Required.

**Mathematical Formulation**

Each element of the output is defined as the result of the following expression:

```python
if x <= input_low:
    output = output_low
elif x > input_high:
    output = output_high
else:
    # input_low < x <= input_high
    output = round((x - input_low) / (input_high - input_low) * (levels-1)) / (levels-1) * (output_high - output_low) + output_low
```

**Example**
```xml
<layer … type="FakeQuantize"…>
    <data levels="2"/>
    <input>
        <port id="0">
            <dim>1</dim>
            <dim>64</dim>
            <dim>56</dim>
            <dim>56</dim>
        </port>
        <port id="1">
            <dim>1</dim>
            <dim>64</dim>
            <dim>1</dim>
            <dim>1</dim>
        </port>
        <port id="2">
            <dim>1</dim>
            <dim>64</dim>
            <dim>1</dim>
            <dim>1</dim>
        </port>
        <port id="3">
            <dim>1</dim>
            <dim>1</dim>
            <dim>1</dim>
            <dim>1</dim>
        </port>
        <port id="4">
            <dim>1</dim>
            <dim>1</dim>
            <dim>1</dim>
            <dim>1</dim>
        </port>
    </input>
    <output>
        <port id="5">
            <dim>1</dim>
            <dim>64</dim>
            <dim>56</dim>
            <dim>56</dim>
        </port>
    </output>
</layer>
```

In [ ]:
device = 'CPU'
ie_avg_fps, predictions = ie_main(IE_MODEL_DEFAULT_INT8_XML, IE_MODEL_DEFAULT_INT8_BIN, device, 'INT8 D')

draw_image(TF_RESULT_IMAGE, predictions, COMBO_RESULT_IMAGE, color=(0, 0, 255))

show_results_interactively(tf_image=TF_RESULT_IMAGE,
                           ie_image=IE_RESULT_IMAGE,
                           combination_image=COMBO_RESULT_IMAGE,
                           ie_fps=ie_avg_fps,
                           tf_fps=tensorflow_average_fps)

show_performance(PERFORMANCE)

In [ ]:
!cat data/configs/default/accuracy_checker_config.yml

In [ ]:
# Replace variables to the real path in the Accuracy Checker config:
!WORKSHOP_PATH=$(pwd) envsubst '\${WORKSHOP_PATH}' <data/configs/default/accuracy_checker_config_template.yml >data/configs/default/accuracy_checker_config.yml

# Run the Accuracy Checker
!accuracy_check -c data/configs/default/accuracy_checker_config.yml

# Section 12: [AccuracyAware Algorithm](https://docs.openvinotoolkit.org/latest/_compression_algorithms_quantization_accuracy_aware_README.html)<a id='s12'></a>

In [ ]:
!python3 ${INTEL_OPENVINO_DIR}/deployment_tools/tools/post_training_optimization_toolkit/main.py \
-c data/configs/accuracy_aware/quantization_config.json \
--output-dir data/public/ssd_mobilenet_v2_coco/INT8/acuracy_aware \
--direct-dump

In [ ]:
! cat data/configs/accuracy_aware/quantization_config.json

1. The model gets fully quantized using the DefaultQuantization algorithm.
2. The quantized and full-precision models are compared on a subset of the validation set in order to find mismatches in the target accuracy metric. A ranking subset is extracted based on the mismatches.
3. A layer-wise ranking is performed in order to get a contribution of each quantized layer into the accuracy drop.
4. Based on the ranking, the most "problematic" layer is reverted back to the original precision. This change is followed by the evaluation of the obtained model on the full validation set in order to get a new accuracy drop.
5. If the accuracy criteria are satisfied for all pre-defined accuracy metrics, the algorithm finishes. Otherwise, it continues reverting the next "problematic" layer.
6. It may happen that regular reverting does not get any accuracy improvement or even worsen the accuracy. Then the re-ranking is triggered as it is described in step 3.

In [ ]:
device = 'CPU'
ie_avg_fps, predictions = ie_main(IE_MODEL_AA_INT8_XML, IE_MODEL_AA_INT8_BIN, device, 'INT8 AA')

draw_image(TF_RESULT_IMAGE, predictions, COMBO_RESULT_IMAGE, color=(0, 0, 255))

show_results_interactively(tf_image=TF_RESULT_IMAGE,
                           ie_image=IE_RESULT_IMAGE,
                           combination_image=COMBO_RESULT_IMAGE,
                           ie_fps=ie_avg_fps,
                           tf_fps=tensorflow_average_fps)

show_performance(PERFORMANCE)

In [ ]:
# Replace variables to the real path in the Accuracy Checker config:
!WORKSHOP_PATH=$(pwd) envsubst '\${WORKSHOP_PATH}' <data/configs/accuracy_aware/accuracy_checker_config_template.yml >data/configs/accuracy_aware/accuracy_checker_config.yml

# Run the Accuracy Checker
!accuracy_check -c data/configs/accuracy_aware/accuracy_checker_config.yml

# Section 13: VNNI - [Deep Learning Boost](https://www.intel.ai/intel-deep-learning-boost/)<a id='s13'></a>

![](pictures/dl_boost.png)

# Section 14: [Get Even Better Performance](https://docs.openvinotoolkit.org/latest/_docs_performance_benchmarks.html)<a id='s14'></a>

Great performance results! But if you want the best performance, use C++.

There is a [C++ benchmark application](https://docs.openvinotoolkit.org/latest/_inference_engine_samples_benchmark_app_README.html) in Inference samples. Let's build it from sources and try!

In [ ]:
!${HOME}/inference_engine_cpp_samples_build/intel64/Release/benchmark_app -h

In [ ]:
!${HOME}/inference_engine_cpp_samples_build/intel64/Release/benchmark_app -h

In [ ]:
!${HOME}/inference_engine_cpp_samples_build/intel64/Release/benchmark_app -m data/public/ssd_mobilenet_v2_coco/FP32/ssd_mobilenet_v2_coco.xml

In [ ]:
PERFORMANCE['OpenVINO IE Benchmark INT8'] = <PLACEHOLDER>
show_performance(PERFORMANCE)

# Section 15: Practice (Part 1)<a id='s15'></a>

Now try to use this model to detect objects in a video. Fill in the gaps in the code blocks below. Refer to the previous cells for hints. For more information about the OpenVINO Inference Engine Python API, see the [official documentation](https://docs.openvinotoolkit.org/latest/ie_python_api/annotated.html).

In [ ]:
import os

# Contains all data for the workshop
WORKSHOP_DATA_PATH = os.path.join('.', 'data')

# Path to the downloaded TensorFlow image
SSD_ASSETS = os.path.join(WORKSHOP_DATA_PATH, 'public', 'ssd_mobilenet_v2_coco')

# Path to the Inference Engine FP32 model
# But you can use the INT8 model instead
MODEL_PATH_XML = os.path.join(SSD_ASSETS, 'FP32', 'ssd_mobilenet_v2_coco.xml')
MODEL_PATH_BIN = os.path.join(SSD_ASSETS, 'FP32', 'ssd_mobilenet_v2_coco.bin')

DEVICE = 'CPU'

DATA_PATH = os.path.join('practice', 'data')
INPUT_VIDEO = os.path.join(DATA_PATH, 'input.mp4')
OUTPUT_VIDEO = os.path.join(DATA_PATH, 'output.MP4')

LABELS_PATH = os.path.join(DATA_PATH, 'coco_labels.txt')

In [ ]:
from IPython.display import HTML


# Show a source video
HTML("""<video width="600" height="400" controls><source src="{}" type="video/mp4"></video>""".format(INPUT_VIDEO))

In [ ]:
def prapare_out_video_stream(input_video_stream):
    width  = int(input_video_stream.get(3))
    height = int(input_video_stream.get(4))
    return cv2.VideoWriter(OUTPUT_VIDEO, cv2.VideoWriter_fourcc(*'X264'), 20, (width, height))

In [ ]:
# Import OpenCV for work with a video and images
import cv2

# Import the Inference Engine
from openvino.inference_engine import IECore, IENetwork

### Step 1: Create an instance of the OpenVINO Inference Engine `IECore` class
This class represents an Inference Engine entity 
and allows you to manipulate plugins using unified interfaces. 

In [ ]:
ie = 

### Step 2: Read the prepared model

The `IENetwork` class is designed to work with a model in the Inference Engine.
This class contains information about the network model read from the Intermediate Representation
and allows you to manipulate some model parameters such as layers affinity and output layers.

You need to create an instance of the IENetwork class.
A constructor of this class has two parameters: 
 1. path to the .xml file of the model 
 2. path to the .bin file of the model

In [ ]:
net =

### Step 3: Get the name of the input layer of the model

To infer a model, you need to know input layers of the model
The object `net` contains information about inputs of the network in a property `inputs`,
which is a dictionary: key - name of the input layer, volume - representation of the input network.
In this case, you need to get only the name of the input. `input_blob` should be a string.

In [ ]:
input_blob =

print('Input layer of the network is {}'.format(input_blob))

### Step 4: Get shape (dimensions) of the input layer of the network

* n - number of batches
* c - number of input image channels (usualy 3 - R, G and B) 
* h - height
* w - width

In [ ]:
n, c, h, w = 

print(f'Input shape of the network: [{n}, {c}, {h}, {w}]'

### Step 5: Get names of output layers of the network

For easier processing of inferences result, we should know names of outputs.
The object `net` contains information about outputs in the same way as inputs.
There is the property `outputs` in the class `IENetwork`.
You need to get the name of the output. `out_blob` should be a string.

In [ ]:
out_blob = 

print(f'Output layer of the network: {out_blob}')

Load names of COCO classes from a file 

In [ ]:
# Load names of COCO classes from a file 
with open(LABELS_PATH, 'r') as f:
    labels_map = [x.strip() for x in f]

### Step 6: Load the network to a device

Use the instance of `IECore`.
The class `IECore` has a special function called `load_network`, which loads a network to a device.
This function prepares the network for the first inference on the device 
and returns an instance of the network prepared for an inference (execution). 
This function has many parameters, but in this case, you need to know only about two of them:
* `network` - instance of `IENetwork`
* `device_name` - string, contains a device name to infer a model on: CPU, GPU and so on.

In [ ]:
network_loaded_to_device = 

### Step 7: Open the input video

In [ ]:
input_video_stream = cv2.VideoCapture(INPUT_VIDEO)

### Step 8: Create an output video stream

In [ ]:
out = prapare_out_video_stream(input_video_stream)

### Step 9: Function for processing inference results

The output layer of the SSD MobileNet V2 is the DetectionOutput layer.
Data layout of this layer (and `obj` variable):
 * element 0 - the batch ID (not important in our case)
 * element 1 - the class ID of a discovered object 
 * element 2 - the confidence for the object
 * element 3 and 4 - coordinates of the upper-left corner of the box of the object
 * element 5 and 6 - coordinates of bottom-right corner of the box of the object

In [ ]:
def draw_boxes_in_frame(obj, frame_w, frame_h, labels_map):
    threshold = 0.0001
        
    # Step 13: Get the confidence for a discovered object
    confidence =
        
    # Step 14: Draw bounding boxes
    # Draw a bounding box only for objects the confidence of which is greater than a specified threshold
    if confidence > threshold:
        # Get coordinates of a discovered object
        xmin =
        ymin =
            
        xmax =
        ymax =
            
        # and scale it to the original size of the frame
        scaled_xmin = int( xmin * frame_w )
        scaled_ymin = int( ymin * frame_h )
            
        scaled_xmax = int( xmax * frame_w )
        scaled_ymax = int( ymax * frame_h )

        # Get class ID of a discovered object
        class_id =

        # Get confidence for a discovered object
        confidence = round(confidence * 100, 1)

        # Draw a box and a label
        color = (min(class_id * 12.5, 255), min(class_id * 7, 255), min(class_id * 5, 255))
        cv2.rectangle(frame, (scaled_xmin, scaled_ymin), (scaled_xmax, scaled_ymax), color, 2)

        # Get the label of a class
        label = labels_map[class_id]

        # Create the title of an object
        text = '{}: {}% '.format(label, confidence)

        # Put the title to a frame
        cv2.putText(frame, text, (scaled_xmin, scaled_ymin - 7), cv2.FONT_HERSHEY_COMPLEX, 2, color, 2)

### Step 10: Loop over frames in the input video

In [ ]:
while input_video_stream.isOpened():
    
    # Read the next frame from the intput video 
    ret, frame = input_video_stream.read()
    # Check if the video is over
    if not ret:
        # Exit from the loop if the video is over
        break 
    # Get height and width of the frame
    frame_h, frame_w = frame.shape[:2]
    
    # Resize the frame to the network input 
    in_frame = cv2.resize(frame, (w, h))
    
    # Change the data layout from HWC to CHW
    in_frame = in_frame.transpose((2, 0, 1))  
    
    # Reshape the frame to the network input 
    in_frame = in_frame.reshape((n, c, h, w))
    
    # To infer the frame, prepare the data.
    # This must be a dictionary: 
    #   key - name of the input layer (you get this early)
    #   value - input data (the prepared frame)  
    feed_dict = {
    }
    
    # All is ready for the main thing - inference!
    # You have read and loaded the network to the device, prepared input data and now you are ready to infer.
    
    # Step 11:
    # To start an inference, call the `infer` function of the `network_loaded_to_device` variable. 
    # We must set input data (a dictionary).
    inference_result = 
    
    # Great! The `inference_result` variable contains output data after inference of the network.
    # `inference_result` is a dictionary, 
    #  where key is the name of the output name, 
    #        value is data from the blob.
    
    # Step 12: Then iterate over all discovered objects
    for obj in inference_result[out_blob][0][0]:
        # You need to run the draw_boxes_in_frame function :
        draw_boxes_in_frame()
    
    # Write the resulting frame to the output stream
    out.write(frame)

# Save the resulting video
out.release()

In [ ]:
from IPython.display import HTML

# Show a source video
HTML("""<video width="600" height="400" controls><source src="{}" type="video/mp4"></video>""".format(OUTPUT_VIDEO))

Do you see boxes in the video? 
If yes, you did all right!
**Good Work!** 

## Section 16: Practice (Part 2)

What is the next step? Often from neural networks build pipelines. It is to use the results of the first neural network as an input for the next neural network. 
Let's try to build a pipeline from two networks:  first is finds a person on the video and the next to recognize the emotions of this person

We have already run the first network. And find the person on the video.
The next step is to find a network for emotion recognition.
There is a good neural network in the [OpenModelZOO](https://docs.openvinotoolkit.org/2019_R1/_docs_Pre_Trained_Models.html) - [emotions-recognition-retail-0003 network](https://docs.openvinotoolkit.org/2019_R1/_emotions_recognition_retail_0003_description_emotions_recognition_retail_0003.html)

### Step 1: Download emotions-recognition-retail-0003 network
Run the Model Downloader eith needed arguments to download the emotions-recognition-retail-0003 network:

In [ ]:
!python3 ${INTEL_OPENVINO_DIR}/deployment_tools/open_model_zoo/tools/downloader/downloader.py -h

This mode already is in OpenVINO format and you do not need to convert it.

After downloading the model you can use it:

### Step 2: Read the prepared model
The IENetwork class is designed to work with a model in the Inference Engine. This class contains information about the network model read from the Intermediate Representation and allows you to manipulate some model parameters such as layers affinity and output layers.

You need to create an instance of the IENetwork class. A constructor of this class has two parameters:

path to the .xml file of the model
path to the .bin file of the model

In [ ]:
emotion_recognition_network = 

### Step 3: Load the network to a device

Use the instance of `IECore`.
The class `IECore` has a special function called `load_network`, which loads a network to a device.
This function prepares the network for the first inference on the device 
and returns an instance of the network prepared for an inference (execution). 
This function has many parameters, but in this case, you need to know only about two of them:
* `network` - instance of `IENetwork`
* `device_name` - string, contains a device name to infer a model on: CPU, GPU and so on.

In [ ]:
emotion_recognition_network_loaded_on_device =

### Step 4: Open the input video

In [ ]:
input_video_stream = cv2.VideoCapture(INPUT_VIDEO)

### Step 5: Create an output video stream

In [ ]:
out = prapare_out_video_stream(input_video_stream)

### Step 6: Prepare a frame and run inference

In [ ]:
def emotion_infer(frame):
    # Find inputs of the model
    em_input_layer = 

     # Get input shape of the network
    n, c, h, w = emotion_recognition_network.inputs[em_input_layer].shape

    # Resize the frame to the network input 
    em_in_frame = cv2.resize(frame, (w, h))
    
    # Change the data layout from HWC to CHW
    em_in_frame = em_in_frame.transpose((2, 0, 1))  
    
    # Reshape the frame to the network input 
    em_in_frame = em_in_frame.reshape((n, c, h, w))
    
    # Find inputs of the model
    em_output_layer = 
    
    # Run the inference how you did it early
    em_results =
    
    # For understanding what is the result of inference this model, check documentation 
    # https://docs.openvinotoolkit.org/latest/_models_intel_emotions_recognition_retail_0003_description_emotions_recognition_retail_0003.html
    return em_results[em_output_layer]

### Step 16: Drow boxes and emotions in a frame

In [ ]:
def draw_boxes_and_emotion_in_frame(obj, frame_w, frame_h, labels_map):
    threshold = 0.0001
        
    # Step 13: Get the confidence for a discovered object
    confidence =
        
    # Step 14: Draw bounding boxes
    # Draw a bounding box only for objects the confidence of which is greater than a specified threshold
    if confidence > threshold:
        # Get coordinates of a discovered object
        xmin =
        ymin =
            
        xmax =
        ymax =
            
        # and scale it to the original size of the frame
        scaled_xmin = int( xmin * frame_w )
        scaled_ymin = int( ymin * frame_h )
            
        scaled_xmax = int( xmax * frame_w )
        scaled_ymax = int( ymax * frame_h )

        # Get class ID of a discovered object
        class_id =

        # If class is person, run inference of emotions-recognition-retail-0003
        if class_id == 1:
            person_image = frame[scaled_ymin:scaled_ymax, scaled_xmin:scaled_xmax]
            # Run emotions-recognition-retail-0003
            emotions_prob = 
            
            emotions = ['neutral', 'happy', 'sad', 'surprise', 'angry']
            emotion = emotions[np.argmax(emotions_prob)]
            
        # Get confidence for a discovered object
        confidence = round(confidence * 100, 1)

        # Draw a box and a label
        color = (min(class_id * 12.5, 255), min(class_id * 7, 255), min(class_id * 5, 255))
        cv2.rectangle(frame, (scaled_xmin, scaled_ymin), (scaled_xmax, scaled_ymax), color, 2)

        # Get the label of a class
        label = labels_map[class_id]

        # Create the title of an object
        text = '{}: {}% '.format(label, confidence)
        text = '{} {}'.format(text, emotion) if class_id == 1 else text
        # Put the title to a frame
        cv2.putText(frame, text, (scaled_xmin, scaled_ymin - 7), cv2.FONT_HERSHEY_COMPLEX, 2, color, 2)

### Step 17: Loop over frames in the input video

In [ ]:
import numpy as np

while input_video_stream.isOpened():
    
    # Read the next frame from the intput video 
    ret, frame = input_video_stream.read()
    # Check if the video is over
    if not ret:
        # Exit from the loop if the video is over
        break 
    # Get height and width of the frame
    frame_h, frame_w = frame.shape[:2]
    
    # Resize the frame to the network input 
    in_frame = cv2.resize(frame, (w, h))
    
    # Change the data layout from HWC to CHW
    in_frame = in_frame.transpose((2, 0, 1))  
    
    # Reshape the frame to the network input 
    in_frame = in_frame.reshape((n, c, h, w))
    
    # To infer the frame, prepare the data.
    # This must be a dictionary: 
    #   key - name of the input layer (you get this early)
    #   value - input data (the prepared frame)  
    feed_dict = {
    }
    
    # All is ready for the main thing - inference!
    # You have read and loaded the network to the device, prepared input data and now you are ready to infer.
    
    # Step 11:
    # To start an inference, call the `infer` function of the `network_loaded_to_device` variable. 
    # We must set input data (a dictionary).
    inference_result = 
    
    # Great! The `inference_result` variable contains output data after inference of the network.
    # `inference_result` is a dictionary, 
    #  where key is the name of the output name, 
    #        value is data from the blob.
    
    # Step 12: Then iterate over all discovered objects
    for obj in inference_result[out_blob][0][0]:
        draw_boxes_and_emotion_in_frame() # you need to run draw_boxes_and_emotion_in_frame with a needed argument

    # Write the resulting frame to the output stream
    out.write(frame)

# Save the resulting video
out.release()

Now the person (Artyom) on the resulting video will be detected with emotion:

In [ ]:
from IPython.display import HTML

# Show a source video
HTML("""<video width="600" height="400" controls><source src="{}" type="video/mp4"></video>""".format(OUTPUT_VIDEO))

![](pictures/thankyou.PNG)